In [ ]:
import os

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

import client as cp
import middleware as mp

# Matplot lib settings
%matplotlib notebook
matplotlib.rcParams.update({
        'font.size': 11,
        'font.family': 'sans-serif',
        'xtick.labelsize': 10,
        'ytick.labelsize': 10
    })

# Numpy settings
np.set_printoptions(precision=2)
np.set_printoptions(suppress=True)

In [ ]:
base_dir = '/Users/ruifengxu/Development/asl-fall16-project/logs/2k/server-{}-replication-{}-write-{}-message-{}/'

In [ ]:
num_servers = 7
for replication_factor in [1, num_servers]:
    for write_percent in [1, 10]:
        for message_size in [128, 1024]:
            print('{:3}R {:3}W {:5}M: '.format(replication_factor, write_percent, message_size), end='')
            for repetition in range(1, 6, 1):
                filename = os.path.join(
                    base_dir.format(num_servers, replication_factor, write_percent, message_size),
                    'repetition-{}-trace.log'.format(repetition)
                )
                print(mp.count_client_connection_time(filename), end = ' ')
            print()

In [ ]:
y = []
y_mean = []
r = 5
for message_size in [128, 1024]:
    for write_percent in [1, 10]:
        for replication_factor in [1, num_servers]:
            print('{:>3}R {:>3}W {:>5}M: '.format(replication_factor, write_percent, message_size), end='')

            config = {
                'request_type': 'a',
                'start': 11,
                'duration': 40,
                'ci_coefficient': 2.776,
                'num_repetitions': 5,
                'num_client_vms': 3,
                'num_clients_per_vm': 110,
                'base_dir': base_dir.format(num_servers, replication_factor, write_percent, message_size)
            }
            client = cp.aggregate_data(config)            
            y.append(client['tps'])
            y_mean.append(np.mean(client['tps']))
            print(client['tp_mean'], client['tp_ci'], 330 / client['tp_mean'] * 1000 / client['rt_mean'])
        print()

In [ ]:
print('y')
for y_single in y:
    print('{:>8.0f} {:>8.0f} {:>8.0f} {:>8.0f} {:>8.0f}'.format(*y_single))
print()

levels = [-1, 1]

print('{:>8} {:>8} {:>8} {:>8} {:>8} {:>8} {:>8} {:>8} {:>8}'.format(
        'I', 'A', 'B', 'C', 'AB',  'AC', 'BC', 'ABC', 'mean'))
for C_index, C in enumerate(levels):
    for B_index, B in enumerate(levels):
        for A_index, A in enumerate(levels):
            index =  C_index * 4 + B_index * 2 + A_index
            print('{:>8} {:>8} {:>8} {:>8} {:>8} {:>8} {:>8} {:>8} {:8.2f} {:>8}'.format(
                    1, A, B, C, A * B,  A * C, B * C, A * B * C, y_mean[index], index))

total = [0] * 8

for C_index, C in enumerate(levels):
    for B_index, B in enumerate(levels):
        for A_index, A in enumerate(levels):
            index =  C_index * 4 + B_index * 2 + A_index
            total[0] += (1 * y_mean[index])
            total[1] += (A * y_mean[index])
            total[2] += (B * y_mean[index])
            total[3] += (C * y_mean[index])
            total[4] += (A * B * y_mean[index])
            total[5] += (A * C * y_mean[index])
            total[6] += (B * C * y_mean[index])
            total[7] += (A * B * C * y_mean[index])
print('{:>8.2f} {:>8.2f} {:>8.2f} {:>8.2f} {:>8.2f} {:>8.2f} {:>8.2f} {:>8.2f}'.format(*total))

# Effects 
qs = np.array(total) / 8
print('{:>8.2f} {:>8.2f} {:>8.2f} {:>8.2f} {:>8.2f} {:>8.2f} {:>8.2f} {:>8.2f}'.format(*qs))

y_hat = []
for C_index, C in enumerate(levels):
    for B_index, B in enumerate(levels):
        for A_index, A in enumerate(levels):
            y_hat_single = qs.dot([1, A, B, C, A * B,  A * C, B * C, A * B * C])
            y_hat.append(y_hat_single)

print()
print('y_hat')
print('{:>8.2f} {:>8.2f} {:>8.2f} {:>8.2f} {:>8.2f} {:>8.2f} {:>8.2f} {:>8.2f}'.format(*y_hat))

print()

print('y_error')
y_error = []
for C_index, C in enumerate(levels):
    for B_index, B in enumerate(levels):
        for A_index, A in enumerate(levels):
            index =  C_index * 4 + B_index * 2 + A_index
            y_error.append(y[index] - y_hat[index])
            print('{:>8.2f} {:>8.2f} {:>8.2f} {:>8.2f} {:>8.2f} '.format(*y_error[index]))

print()
print('Sum of error {:>8.1f}'.format(np.sum(y_error)))
print()


SSY = np.sum(np.power(y, 2))
print('SSY', SSY)

SS0 = np.power(2, 3) * r * qs[0] * qs[0]
print('SS0', SS0)

SST = SSY - SS0
print('SST', SST)

SS = []
for q in qs:
    SS.append(8 * r * q * q)
SS
print('SS')
print('{:>8.2f} {:>8.2f} {:>8.2f} {:>8.2f} {:>8.2f} {:>8.2f} {:>8.2f} {:>8.2f}'.format(*list(SS)))

SSE = SST - np.sum(SS) + SS[0]
print('SSE', SSE)

print('Variation')
print('{:>8.5f} {:>8.5f} {:>8.5f} {:>8.5f} {:>8.5f} {:>8.5f} {:>8.5f} {:>8.5f}'.format(*list(np.array(SS) / SST)))

print('error variation {:>8.5f}'.format(SSE / SST))

SE2 = SSE / (8 * (r - 1))
print('SE2', SE2)

SQS = np.sqrt(SE2 / (8 * r))
print('SQS', SQS)

print(2.036 * SQS)

print(qs)
print(np.array(qs) - (2.036 * SQS))
print(np.array(qs) + (2.036 * SQS))

np.min(y)

In [ ]:
py = []
for i in y_error:
    for j in i:
        py.append(j)

px = []
for i in y_hat:
    for j in range(0, 5):
        px.append(i)
        
fig = plt.figure(figsize=(10, 4))   
plt.subplot(1, 2, 1)
plt.scatter(px, py)
plt.ylabel('Residual')
plt.axhline(0, color='black')
plt.xlabel('Predicted response')
plt.title('(a)')
plt.xlim([0, 20000])

from scipy.stats import probplot
plt.subplot(1, 2, 2)
probplot(py, dist="norm", plot=plt)
plt.xlabel('Normal quantile')
plt.ylabel('Residual quantile')
plt.title('(b)')
plt.tight_layout()
fig.savefig('2k.png', dpi=300, bbox_inches='tight', pad_inches=0)

In [ ]:
SSE = 0
for i in y_error:
    for j in i:
        SSE += j * j        
print('SSE', SSE)

y_all = []
for y_single in y:
    y_all += list(y_single)
y_all_mean = np.mean(y)
print('y_all_mean', y_all_mean)

SSY = 0
for item in y_all:
    SSY += item * item
print('SSY', SSY)

SST = 0
for item in y_all:
    SST += (item - y_all_mean) * (item - y_all_mean)
print('SST', SST)

SS = []
for q in qs:
    SS.append(8 * r * q * q)
SS
print('SS', SS)
    
print('SST`', SSY - SS[0])

print('SSE`', SSY - np.sum(SS))
